In [19]:
## To use Google Colab

# from google.colab import drive
# drive.mount("/gdrive", force_remount=True)
# # drive.mount('/gdrive')


Mounted at /gdrive


In [1]:
# %cd /gdrive
# %cd 'My Drive'
# # %cd Maga
# %cd Diploma

In [3]:
!ls

morewiki_1456.json  morewiki_50.json
morewiki_20.json    ranking_experiments.ipynb


In [0]:
# !git clone https://github.com/huggingface/transformers.git
# # !git clone https://github.com/huggingface/transformers --branch quickstart-model2model
# %cd transformers
# !pip3 install .
# !git pull
# !pip3 install --upgrade .

In [0]:
# ! pip3 uninstall transformers

In [2]:
# # !git clone https://github.com/huggingface/transformers
# %cd transformers
# !pip3 install .

In [24]:
import json
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [25]:
# !tar -xzf rubert_cased_L-12_H-768_A-12_pt.tar.gz

In [26]:
# import os
# os.chdir("../")

In [27]:
# !ls
# !cd rubert_cased_L-12_H-768_A-12_pt

In [28]:
# ru_bert_path = "rubert_cased_L-12_H-768_A-12_pt"
ru_bert_path = "/datadrive/axon/something_reasoning/models/rubert_cased_L-12_H-768_A-12_pt/"

In [29]:
relations = ["синонимы", "labels", "sitelinks", "summary", "content"]

In [30]:
tokenizer = BertTokenizer.from_pretrained(ru_bert_path)

In [31]:
model = BertModel.from_pretrained(ru_bert_path)

In [32]:
model = model.eval()

In [33]:
from tqdm import tqdm

In [34]:
def default_collate(batch):
    return {k: torch.LongTensor(v) for k, v in batch.items()}

In [35]:
def get_vec(text):

    # print("text", text)

    b = default_collate(tokenizer.batch_encode_plus([text]))
    out = model(**b)

    # print("b", b)
    # print("out", out)
    
    vec = out[1][0]

    # print("vec", vec)

    return vec.detach().numpy()

In [36]:
def get_cosins(query_text, info):
    query = get_vec(query_text)
    
    example = []
    
    # count = 0

    for q in info["ranking_candidates"]:

        vectors = []

        for val in q["graph"]: # TypeError: string indices must be integers

            if val["relation"] in relations:
                text = val["value"] if isinstance(val["value"], str) else " ".join(val["value"])
                text = text[:256]
                val_vec = get_vec(text)
                vectors.append(val_vec)
        # for doc in q["documents"]:
        #     text = doc["document"][:256]
        #     val_vec = get_vec(text)
        #     vectors.append(val_vec)
        v = np.vstack(vectors)
        qv = np.vstack([query] * len(v))

        # print("cosine_similarity(v, qv)[:, 0]", cosine_similarity(v, qv)[:, 0])
        # print("cosine_similarity(v, qv)", cosine_similarity(v, qv))

        example.append({"id": q["Q"], "cosins": cosine_similarity(v, qv)[:, 0]})
    return example

In [37]:
# # wikilink_final/morewiki_20.json
# with open("morewiki_1456.json", "r") as file:
#     data = json.load(file)

In [38]:
with open("all_wiki_1810r21rubert_nerdp.json", "r") as file:
    data = json.load(file)

In [39]:
from tqdm import tqdm_notebook

In [40]:
def get_cosins_all(data):
    lines = []
    y_trues = []
    questions = [] # for getting ranked Qs
    
    for query_text in tqdm_notebook(data, total=len(data), leave=False):
        info = data[query_text]
        y_trues.append(info.get("y_gold"))

        # change for NER
        ner = data[query_text]["nerdp"]
        lines.append(get_cosins(ner, info))
        
#         lines.append(get_cosins(query_text, info))

        questions.append(query_text) # for Qs
    
    return lines, y_trues, questions

In [41]:
def get_scores(lines, y_trues, top_n=2):
    answers = 0
    all_counter = 0
    zero_counts = 0
    lens_ranking = []

    for line, y_true in zip(lines, y_trues):
        cosins = []
        ids = []
        for q in line:
            cosins.append(max(q["cosins"]))
            ids.append(q["id"])
        lens_ranking.append(len(cosins))
        if cosins:
            idx = np.argsort(cosins)[-top_n:] # ?
            if y_true is not None:
                all_counter += 1
                if y_true in np.array(ids)[idx]:
                    answers += 1
        else:
            zero_counts += 1
    return answers, all_counter, zero_counts, lens_ranking

In [42]:
def get_top_5(lines, y_trues, questions, top_n=2):
    answers = 0
    all_counter = 0
    zero_counts = 0
    lens_ranking = []
    question_candidates = {}

    for line, y_true, question in zip(lines, y_trues, questions):
        cosins = []
        ids = []
        for q in line:
            cosins.append(max(q["cosins"]))
            ids.append(q["id"])
        lens_ranking.append(len(cosins))
        if cosins:
            idx = np.argsort(cosins)[-top_n:]
            if y_true is not None:
                all_counter += 1
                top_Q = np.array(ids)[idx]
                question_candidates[question] = top_Q
                if y_true in np.array(ids)[idx]:
                    answers += 1
        else:
            zero_counts += 1
    return answers, all_counter, zero_counts, lens_ranking, question_candidates

In [43]:
# with open("ranking_data_v1-1.json", "r") as file:
#     data = json.load(file)

In [44]:
# with open("res4_wikidata.json", "r") as file:
#     data = json.load(file)

In [45]:
# !ls

In [19]:
lines, y_trues, questions = get_cosins_all(data)

In [22]:
answers, all_counter, zero_counts, lens_ranking, question_candidates = get_top_5(lines, y_trues, questions, top_n=21)


In [23]:
for q in question_candidates:
    if type(question_candidates[q]) == list:
        continue
    question_candidates[q] = question_candidates[q].tolist()
#     print(question_candidates[q])
#     break

In [46]:
import json

with open("Q_for_gr_ranked_ner_dp.json", "w", encoding="utf-8") as f:
        json.dump(question_candidates, f, ensure_ascii=False, indent=4)

## For 1800 questions, 1-21 Qs per question to rank -- with Deep Pavlov NER 

In [47]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=1)

In [48]:
answers / len(lines)

0.6812154696132596

In [49]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=2)

In [50]:
answers / len(lines)

0.8259668508287292

In [51]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=3)

In [52]:
answers / len(lines)

0.8740331491712707

### For 1810 questions, 1-21 Qs per question to rank -- NER ranked

In [116]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=1)

In [117]:
answers / len(lines)

0.9788888888888889

In [118]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=2)

In [119]:
answers / len(lines)

0.9905555555555555

In [120]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=3)

In [121]:
answers / len(lines)

0.9944444444444445

In [122]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=5)

In [123]:
answers / len(lines)

0.9977777777777778

### For 1810 questions, 1-21 Qs per question to rank

In [81]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=1)

In [82]:
answers / len(lines)

0.48066298342541436

In [83]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=2)

In [84]:
answers / len(lines)

0.6591160220994475

In [85]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=3)

In [86]:
answers / len(lines)

0.7828729281767955

In [87]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=5)

In [88]:
answers / len(lines)

0.8823204419889503

### For 1456 questions, 2-21 Qs per question to rank
(morewiki_1456.json)

In [38]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=1)

In [39]:
answers / len(lines)

0.3543956043956044

In [40]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=2)

In [41]:
answers / len(lines)

0.5762362637362637

In [42]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=3)

In [43]:
answers / len(lines)

0.7300824175824175

In [45]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=5)

In [46]:
answers / len(lines)

0.8537087912087912

In [47]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=6)

In [48]:
answers / len(lines)

0.885989010989011

In [59]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=8)

In [60]:
answers / len(lines)

0.9333791208791209

In [44]:
np.mean(lens_ranking)

6.3282967032967035

In [62]:
import numpy as np
np.array([[1,2,3],[4,5,6]])#.tolist()
# [[1, 2, 3], [4, 5, 6]]

array([[1, 2, 3],
       [4, 5, 6]])